In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/datases3/train_identity.csv
/kaggle/input/datases3/train_transaction.csv


Cleaning

In [6]:

# Load and merge data
df1 = pd.read_csv('/kaggle/input/datases3/train_transaction.csv')
df2 = pd.read_csv('/kaggle/input/datases3/train_identity.csv')
df = pd.merge(df1, df2, on='TransactionID', how='left')

# Target
y = df['isFraud']
X = df.drop(['isFraud'], axis=1)

# Drop high-null columns
nan_ratio = X.isnull().mean()
X = X.loc[:, nan_ratio < 0.9]

# Fill missing values
for col in X.select_dtypes(include='number'):
    X[col] = X[col].fillna(X[col].median())
for col in X.select_dtypes(include='object'):
    X[col] = X[col].fillna('missing')


In [3]:
X

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,86400,68.50,W,13926,361.0,150.0,discover,142.0,credit,...,missing,24.0,missing,missing,missing,missing,missing,missing,missing,missing
1,2987001,86401,29.00,W,2755,404.0,150.0,mastercard,102.0,credit,...,missing,24.0,missing,missing,missing,missing,missing,missing,missing,missing
2,2987002,86469,59.00,W,4663,490.0,150.0,visa,166.0,debit,...,missing,24.0,missing,missing,missing,missing,missing,missing,missing,missing
3,2987003,86499,50.00,W,18132,567.0,150.0,mastercard,117.0,debit,...,missing,24.0,missing,missing,missing,missing,missing,missing,missing,missing
4,2987004,86506,50.00,H,4497,514.0,150.0,mastercard,102.0,credit,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,3577535,15811047,49.00,W,6550,361.0,150.0,visa,226.0,debit,...,missing,24.0,missing,missing,missing,missing,missing,missing,missing,missing
590536,3577536,15811049,39.50,W,10444,225.0,150.0,mastercard,224.0,debit,...,missing,24.0,missing,missing,missing,missing,missing,missing,missing,missing
590537,3577537,15811079,30.95,W,12037,595.0,150.0,mastercard,224.0,debit,...,missing,24.0,missing,missing,missing,missing,missing,missing,missing,missing
590538,3577538,15811088,117.00,W,7826,481.0,150.0,mastercard,224.0,debit,...,missing,24.0,missing,missing,missing,missing,missing,missing,missing,missing


Feature Engineering


In [7]:
from sklearn.preprocessing import LabelEncoder

# Label encode object features
for col in X.select_dtypes(include='object').columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))

# Create feature: ratio of transaction to mean in card1 group
X['TransactionAmt_to_mean_card1'] = X['TransactionAmt'] / (X.groupby('card1')['TransactionAmt'].transform('mean') + 1)


/tmp/ipykernel_31/2952385606.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X['TransactionAmt_to_mean_card1'] = X['TransactionAmt'] / (X.groupby('card1')['TransactionAmt'].transform('mean') + 1)


In [5]:
X

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,TransactionAmt_to_mean_card1
0,2987000,86400,68.50,4,13926,361.0,150.0,1,142.0,1,...,24.0,260,4,2,2,2,2,1,1735,0.194089
1,2987001,86401,29.00,4,2755,404.0,150.0,2,102.0,1,...,24.0,260,4,2,2,2,2,1,1735,0.123251
2,2987002,86469,59.00,4,4663,490.0,150.0,4,166.0,2,...,24.0,260,4,2,2,2,2,1,1735,0.601945
3,2987003,86499,50.00,4,18132,567.0,150.0,2,117.0,2,...,24.0,260,4,2,2,2,2,1,1735,0.401876
4,2987004,86506,50.00,1,4497,514.0,150.0,2,102.0,1,...,32.0,164,3,1,0,1,1,2,954,0.510349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,3577535,15811047,49.00,4,6550,361.0,150.0,4,226.0,2,...,24.0,260,4,2,2,2,2,1,1735,0.500021
590536,3577536,15811049,39.50,4,10444,225.0,150.0,2,224.0,2,...,24.0,260,4,2,2,2,2,1,1735,0.162524
590537,3577537,15811079,30.95,4,12037,595.0,150.0,2,224.0,2,...,24.0,260,4,2,2,2,2,1,1735,0.256333
590538,3577538,15811088,117.00,4,7826,481.0,150.0,2,224.0,2,...,24.0,260,4,2,2,2,2,1,1735,0.945350


Feature selection 


In [8]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif

X_numeric = X.select_dtypes(include='number')
selector = SelectKBest(score_func=mutual_info_classif, k=100)
X_selected = selector.fit_transform(X_numeric, y)
selected_features = X_numeric.columns[selector.get_support()]
X = X[selected_features]


In [1]:
!pip install dagshub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.1/260.1 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: dacite
    Found existing installation: dacite 1.9.2
    Uninstalling dacite-1.9.2:
      Successfully uninstalled dacite-1.9.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydata-profiling 4.16.1 requires dacite>=1.8, but you have dacite 1.6.0 which is incompatible.


In [2]:
import dagshub
dagshub.init(repo_owner='nurch22', repo_name='ML2', mlflow=True)


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=3f0a3b2c-b2ed-4238-b55a-7c13a977d1de&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=686ad9cf051dedf77df37bc77a909356d473fe245f1116194df1ee841ed912b8




Output()

Accessing as nurch22

Initialized MLflow to track repo "nurch22/ML2"

Repository nurch22/ML2 initialized!

In [3]:
!pip install mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.2/28.2 MB 52.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 77.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 692.3/692.3 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.6 MB/s eta 0:00:00


Training pliss mlflow logging

In [11]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import mlflow
import mlflow.lightgbm

# Split dataset
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# MLflow tracking
from lightgbm import early_stopping, log_evaluation

model = lgb.LGBMClassifier(
    objective="binary",
    learning_rate=0.05,
    num_leaves=64,
    feature_fraction=0.8,
    bagging_fraction=0.8,
    bagging_freq=5,
    n_estimators=1000,
    random_state=42,
)

mlflow.set_experiment("LightGBM_FraudDetection")

with mlflow.start_run():
    mlflow.log_params(model.get_params())

    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric="auc",
        callbacks=[early_stopping(stopping_rounds=50), log_evaluation(100)]
    )

    preds = model.predict_proba(X_val)[:, 1]
    auc = roc_auc_score(y_val, preds)
    mlflow.log_metric("val_auc", auc)

    mlflow.lightgbm.log_model(model, "model")

    print(f"Validation AUC: {auc:.4f}")



[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Info] Number of positive: 16530, number of negative: 455902
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.194198 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[Lig

2025/04/23 09:36:32 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Validation AUC: 0.8522
🏃 View run calm-auk-903 at: https://dagshub.com/nurch22/ML2.mlflow/#/experiments/3/runs/d070c4fe8326440ab0722b8e5f29e38d
🧪 View experiment at: https://dagshub.com/nurch22/ML2.mlflow/#/experiments/3
